In [10]:
import pandas as pd
import json
from utils import *
import google.generativeai as genai

/Users/sitewang/.pyenv/versions/3.11.9/envs/journal/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
OPENWEATHER_API_KEY = os.getenv('OPENWEATHER_API_KEY')
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))

In [12]:
with open('converted_data/long_journal_unconverted.json', 'r') as file:
    raw = json.load(file)

In [13]:
raw['journalEntries'][0]['date']

'July 4, 2016'

In [14]:
def generate_title(location_data, weather_data, music_data, journal_data):
    
    prompt = f"""
    My name is Long Cheng. I work in the tech industry and is a AI enthusiastic. 
    
    This journal was written at: 
    {location_data}

    Here are some weather information at the time:
    {weather_data}

    I feel the best music to describe this journal is:
    {music_data}

    Here is the journal I wrote:
    {journal_data}
    
    Based on the all above information and the picture provided, can you write a personal journal. 
    Don't mention anything related to technical data provided above.
    Understand what is in the pictutre in the journal. It is okay to keep the imagination open. 
    Be creative, personal, and realistic. 
    """
    vision_model = genai.GenerativeModel('gemini-pro')
    response = vision_model.generate_content(prompt)
    return response.text

In [16]:
journal = {}

for v in raw['journalEntries']:
    date_str = v['date']
    weather_str = v['weather']
    location_str = v['location']
    music_str = v['music']
    entry_str = v['entry']

    latitude, longitude = locate_address_google_map(location_str)
    utc_dt = convert_raw_date_to_utc_dt(date_str, latitude, longitude)
    location_data = get_location_by_coordinates(latitude, longitude)
    openweather_response = get_historical_weather(latitude, longitude, utc_dt, OPENWEATHER_API_KEY)
    weather_data = openweather_response.data.loc[0]
    timezone = openweather_response.timezone.loc[0]
    journal[date_str] = {
        'journal': entry_str,
        'title': generate_title(location_data, weather_data, music_str, entry_str),
        'utc_dt': utc_dt,
        'music': music_str,
        'timezone': timezone,
        'weather_data': weather_data,
        'location_data': location_data
    }

with open('long_journal.json', 'w') as f:
    json.dump(journal, f)